In [ ]:
#     В ЭТОМ КОДЕ:

# вытащить информацию о ЗАЯВКАХ КАНДИДАТОВ за 1-3 созывы

In [1]:
import requests
from bs4 import Tag, NavigableString, BeautifulSoup
import re
from datetime import datetime, date, time

import pandas as pd


In [2]:
# 1. количество тем
# <meta name="description" content="Обсуждения сообщества Студенческий совет мехмата МГУ — 144 темы." />

vk_group_id = '33327655'
thread_link = 'https://vk.com/board' + vk_group_id

req_link = requests.get(thread_link)
html_doc = req_link.text
soup = BeautifulSoup(html_doc, 'html.parser')
meta = soup.find('meta', attrs={'name': 'description'})

threads_cnt = int(re.findall(r' ([\d]+) ', meta['content'])[-1])
# threads_cnt


In [3]:
# список ссылок на треды и число сообщений в них

# про каждый тред:
# <div class="topic_item">
#   <a class="ti_title" href="/topic-33327655_30624962" rel="noopener">[2014, осень], Повышенные стипендии</a>
#   <div class="ti_count"><b>7</b> сообщений</div>
#   <a class="ti_info" href="/topic-33327655_30624962?offset=last#last" rel="noopener">Алексей Матвеев ответил 7 окт 2014</a>
# </div>

def vk_threads_list_by_link(link_vk):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    topics = soup.find_all('div', attrs={'class': 'topic_item'})
    
    thread_dict = {}
    
    for element in topics:
        title = element.find('a', attrs={'class': 'ti_title'})
        thread_link = title['href']
        text_title = [line for line in title.stripped_strings]
        thread_title = text_title[0]
        
        count = element.find('div', attrs={'class': 'ti_count'})
        text_count = [line for line in count.stripped_strings]
        thread_message_count = int(text_count[0])
        
        info = element.find('a', attrs={'class': 'ti_info'})
        text_info = [line for line in info.stripped_strings]
        thread_last_info = text_info[0]
        
#         print(thread_link, thread_title, thread_message_count, thread_last_info)
        
        thread_dict[thread_link] = {
            'thread_link': 'https://vk.com' + thread_link,
            'thread_title': thread_title,
            'thread_message_count': thread_message_count,
            'thread_last_info': thread_last_info
                             }
    
    return thread_dict


In [4]:
threads_dict = {}

for sheet in range(threads_cnt//20 + 1):
    threads_dict_delta = vk_threads_list_by_link('https://vk.com/board33327655' + '?offset=' + str(sheet*20))
    threads_dict = {**threads_dict, **threads_dict_delta}
    
threads_dict

{'/topic-33327655_47378210': {'thread_link': 'https://vk.com/topic-33327655_47378210',
  'thread_title': 'Протоколы собраний',
  'thread_message_count': 8,
  'thread_last_info': 'მომხმარებელმა ადმინისტრატორი უპასუხა 19 აპრ  12:19–ზე'},
 '/topic-33327655_48164294': {'thread_link': 'https://vk.com/topic-33327655_48164294',
  'thread_title': 'Расписание',
  'thread_message_count': 2,
  'thread_last_info': 'მომხმარებელმა ადმინისტრატორი უპასუხა 6 თებერვ  9:13–ზე'},
 '/topic-33327655_26170925': {'thread_link': 'https://vk.com/topic-33327655_26170925',
  'thread_title': '"БЫТОВЫЕ ВОПРОСЫ" - рабочая группа',
  'thread_message_count': 89,
  'thread_last_info': 'მომხმარებელმა ალინა არსლანოვა უპასუხა 10\xa0ოქტ\xa02019–ზე'},
 '/topic-33327655_35904375': {'thread_link': 'https://vk.com/topic-33327655_35904375',
  'thread_title': '[2017, Осень], повышенные стипендии',
  'thread_message_count': 11,
  'thread_last_info': 'მომხმარებელმა ალეკსეი კრემერი უპასუხა 12\xa0სექ\xa02017–ზე'},
 '/topic-33327655_

In [101]:
# ОСТАВЛЯЕМ ТОЛЬКО СО СЛОВОМ 'Кандидаты' в thread_title

election_threads_dict = {}

for kek in threads_dict.values():
    if 'Кандидаты' in kek['thread_title']:
        election_threads_dict[kek['thread_link']] = {
            'title': kek['thread_title'],
            'thread_message_count': kek['thread_message_count']
        }
election_threads_dict


{'https://vk.com/topic-33327655_29117899': {'title': 'Кандидаты 2 курс',
  'thread_message_count': 7},
 'https://vk.com/topic-33327655_29117864': {'title': 'Кандидаты 3 курс',
  'thread_message_count': 9},
 'https://vk.com/topic-33327655_29117929': {'title': 'Кандидаты 1 курс',
  'thread_message_count': 12},
 'https://vk.com/topic-33327655_29117826': {'title': 'Кандидаты 4 курс',
  'thread_message_count': 11},
 'https://vk.com/topic-33327655_29117817': {'title': 'Кандидаты 5 курс',
  'thread_message_count': 9},
 'https://vk.com/topic-33327655_29117788': {'title': 'Кандидаты Магистратура 1 курс',
  'thread_message_count': 3},
 'https://vk.com/topic-33327655_29117770': {'title': 'Кандидаты Магистратура 2 курс',
  'thread_message_count': 2},
 'https://vk.com/topic-33327655_27333029': {'title': 'Кандидаты 1 курс',
  'thread_message_count': 7},
 'https://vk.com/topic-33327655_27333032': {'title': 'Кандидаты 2 курс',
  'thread_message_count': 7},
 'https://vk.com/topic-33327655_27333034': {'

In [ ]:
'''
ДАЛЕЕ: ПРИТЯНУТЬ ДАТУ К ССЫЛКЕ ТРЕДА, ЧТОБЫ ПОТО РАССОРТИРОВАТЬ ПО СОЗЫВАМ
'''

In [6]:
# attr = 'a'
# class_name = 'item_date'

def vk_thread_date(pi_info, attr, class_name, language):
    months_dict = {
        'russian': ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'],
        'english': ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
#         https://en.wikipedia.org/wiki/Georgian_calendar
        'georgian': ['იან', 'თებ', 'მარ', 'აპრ', 'მაი', 'ივნ', 'ივლ', 'აგვ', 'სექ', 'ოქტ', 'ნოე', 'დეკ']
    }
    months = months_dict[language]
    if pi_info:
        wi_date = pi_info.find(attr, attrs={'class': class_name})
        date_list = [date for date in wi_date.stripped_strings]
        link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
        link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
        link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
        link_date = date(link_year_no, link_month_no, link_day_no)
        link_date_time = date_list[0]
        link_time = re.findall(r'[\d]+\:[\d]+', date_list[0])[0]
    else:
        link_year_no = 0
        link_month_no = 0
        link_day_no = 0
        link_date = date(1970, 1, 1)
        link_date_time = ''
        link_time = ''
        
    return link_year_no, link_month_no, link_day_no, link_date, link_date_time, link_time

In [7]:
# СЛОВАРИК ИНФОРМАЦИИ ИЗ ТРЕДА (считывает по 20 постов)

#       <div class="pi_cont">
#         <div class="pi_head">
#               <div><a class="pi_author" href="/am_tropin" rel="noopener">Александр Тропин</a> </div>
#         </div>
#         <div class="pi_body">
#                 <div class="pi_text">На собрании присутствовало 9 представителей студсовета. Модерировал собрание Никита Котл

# <div class="pi_info"><a class="item_date" href="/topic-33327655_25816532?post=104#post104" rel="noopener">10 янв 2012 в 21:49</a></div>
    
def vk_thread_info_by_link(link_vk, language):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ ПОСТА ПОСТРОЧНО    
    # <div class="pi_body">
    #     <div class="pi_text">
    
    pi_body = soup.find_all('div', attrs={'class': 'pi_cont'})
#     print(pi_body)
    
    dict_el = {}
    
    for el in pi_body:
        text = el.find('div', attrs={'class': 'pi_text'})
#         print(text)
        if text:
            text_lines = [line for line in text.stripped_strings]
        else:
            text_lines = []
#         date = pi_info.find('a', attrs={'class': 'item_date'})
        link_year_no, link_month_no, link_day_no, link_date, link_date_time, link_time = vk_thread_date(el, 'a', 'item_date', language)
    
        author = el.find('a', attrs={'class': 'pi_author'})
        author_line = [line for line in author.stripped_strings]
        
        dict_el[pi_body.index(el)] = {
#             'post_no': pi_body.index(el),
            'link_date': link_date, 
            'link_year_no': link_year_no, 
            'link_month_no': link_month_no, 
            'link_day_no': link_day_no, 
            'link_date_time': link_date_time, 
            'link_time': link_time,
            'author': author_line[0],
            'list_of_lines': text_lines
                             }

    dict_link = {}
    dict_link[link_vk] = {
        'list_of_posts': dict_el
                         }
    
    return dict_link


In [78]:
# vk_thread_info_by_link('https://vk.com/topic-33327655_29117899', 'georgian')

In [51]:
# print(vk_thread_info_by_link('https://vk.com/topic-33327655_26170925'))
# print(vk_thread_info_by_link('https://vk.com/topic-33327655_47378210', 'georgian'))


In [102]:
for link in election_threads_dict.keys():
    test_dict = vk_thread_info_by_link(link, 'georgian')
    election_threads_dict[link]['link_date'] = test_dict[link]['list_of_posts'][0]['link_date']
    election_threads_dict[link]['link_year_no'] = test_dict[link]['list_of_posts'][0]['link_year_no']
    election_threads_dict[link]['link_month_no'] = test_dict[link]['list_of_posts'][0]['link_month_no']
    election_threads_dict[link]['link_day_no'] = test_dict[link]['list_of_posts'][0]['link_day_no']
#     print(test_dict[link]['list_of_posts'][0]['link_date'])

election_threads_dict

{'https://vk.com/topic-33327655_29117899': {'title': 'Кандидаты 2 курс',
  'thread_message_count': 7,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17},
 'https://vk.com/topic-33327655_29117864': {'title': 'Кандидаты 3 курс',
  'thread_message_count': 9,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17},
 'https://vk.com/topic-33327655_29117929': {'title': 'Кандидаты 1 курс',
  'thread_message_count': 12,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17},
 'https://vk.com/topic-33327655_29117826': {'title': 'Кандидаты 4 курс',
  'thread_message_count': 11,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17},
 'https://vk.com/topic-33327655_29117817': {'title': 'Кандидаты 5 курс',
  'thread_message_count': 9,
  'link_date': datetime.date(2013, 1

In [9]:
def convocation_no_by_date(link_date):
    if link_date.year == 2013:
        return 3
    elif link_date.year == 2012 and link_date.month >= 9:
        return 2
    elif link_date.year == 2012 and link_date.month <= 5:
        return 1
    else: 
        return 0
    
def year_no_by_title(title):
    if 'магистр' in str.lower(title):
        if re.sub('\D', '', title):
            return 'm'+re.sub('\D', '', title)
        else:
            return 'm1'
    else:
        return re.sub('\D', '', title)
    
# title = 'Кандидаты Магистратура 1 курс'
# title = 'Кандидаты Магистратура 2 курс'
# title = 'Кандидаты 1 курс'
# title = 'Кандидаты, магистратура'

# year_no_by_title(title)

In [103]:
# добавить номер созыва и номер курса

for link in election_threads_dict.keys():
    election_threads_dict[link]['convocation_no'] = convocation_no_by_date(election_threads_dict[link]['link_date'])
    election_threads_dict[link]['year_no'] = year_no_by_title(election_threads_dict[link]['title'])    
    
election_threads_dict

{'https://vk.com/topic-33327655_29117899': {'title': 'Кандидаты 2 курс',
  'thread_message_count': 7,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17,
  'convocation_no': 3,
  'year_no': '2'},
 'https://vk.com/topic-33327655_29117864': {'title': 'Кандидаты 3 курс',
  'thread_message_count': 9,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17,
  'convocation_no': 3,
  'year_no': '3'},
 'https://vk.com/topic-33327655_29117929': {'title': 'Кандидаты 1 курс',
  'thread_message_count': 12,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17,
  'convocation_no': 3,
  'year_no': '1'},
 'https://vk.com/topic-33327655_29117826': {'title': 'Кандидаты 4 курс',
  'thread_message_count': 11,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17,
  'convocation_n

In [253]:
'''
КОСТЫЛЬНЫЙ ПАТТЕРН
'''

# возвращает имя без цифр и знаков препинания
# "/(^(^A|^B))/" = A && B
# '([^(\W|\d)\s]+)'

# (символ И НЕ цифра) ИЛИ пробел == НЕ (НЕ символ ИЛИ цифра) ИЛИ пробел == ^(^символ ИЛИ цифра) ИЛИ пробел == [^([\W\d])\s]
# ([\w\s\-]+)
# r'([[^\d\W]\s]+)'
# '([\w\s]+)' - везде ок кроме '422 - Чернодед Антон Игоревич'
# ([а-яА-Я\s]+)

def clear_name(line):
    line.replace(' - ', ' ')
    nomination_pattern = r'([^\d\,\-\.\:\;]+)'    # [^\d\W]+
#     print(re.findall(nomination_pattern, line))
    if len(re.findall(nomination_pattern, line)) > 0:
        ss = [f.strip() for f in re.findall(nomination_pattern, line) if f.strip() != '']
#         print(ss)
#         print('kek')
        return ss[0].strip()
    else:
#         print('lol')
        return line
    
# line = 'fffffff - 444'
# line = 'fffffff, uhhh 444'
# line = '444 - fffffff' # WHYYYY?
# line = 'fffffff.'
# line = 'Владыкина Вероника Евгеньевна.'
# line = 'Остроухова Наталья Владимировна, 101 группа'
# line = '422 - Чернодед Антон Игоревич'
# clear_name(line)


'Чернодед Антон Игоревич'

In [ ]:
# ТЕПЕРЬ ВЫТАЩИТЬ ВЕСЬ ТЕКСТ ПО ВСЕМ ССЫЛКАМ ИЗ election_threads_dict

In [90]:
# election_threads_dict['https://vk.com/topic-33327655_26089953'][]

In [104]:
# один кандидат - один пост в треде

for link in election_threads_dict.keys():
# link = 'https://vk.com/topic-33327655_29117899'
# if link == 'https://vk.com/topic-33327655_29117899':
    print(election_threads_dict[link]['title'], election_threads_dict[link]['link_date'])
    list_of_lines_0 = []
    list_of_candidates = []
    election_threads_dict_0 = {}
    for sheet in range(election_threads_dict[link]['thread_message_count']//20 + 1):
        list_of_candidates = vk_thread_info_by_link(link + '?offset=' + str(sheet*20), 'georgian')[link + '?offset=' + str(sheet*20)]['list_of_posts'][0]['list_of_lines']
        if election_threads_dict[link]['thread_message_count'] > 1:
            for post_no in range(1, election_threads_dict[link]['thread_message_count']):
#                 условие для удаления хвоста комментов в треде
                if (not (link == 'https://vk.com/topic-33327655_26089953' and post_no >= 8)
                    and not (link == 'https://vk.com/topic-33327655_26089854' and post_no >= 3)
                   ):
                    print(post_no)
                    text = vk_thread_info_by_link(link + '?offset=' + str(sheet*20), 'georgian')[link + '?offset=' + str(sheet*20)]['list_of_posts'][post_no]['list_of_lines']
#                     условие для удаления вводного предложения
                    if (link == 'https://vk.com/topic-33327655_26089953' and post_no == 1):
                        text = text[1:]
                    dict_by_name = {}
                    dict_by_name[text[0]] = text[1:]
                    election_threads_dict_0 = {**election_threads_dict_0, **dict_by_name}
        else:
            election_threads_dict_0 = {}
    election_threads_dict[link]['list_of_candidates'] = list_of_candidates
    election_threads_dict[link]['candidates'] = election_threads_dict_0

election_threads_dict

# ВНИМАНИЕ: ЕЩЕ ТАМ ЕСТЬ ПОСТЫ С РЕЗУЛЬТАТАМИ ВЫБОРОВ
# а еще есть комменты от других студентов

# баги:
#     33327655_27333041 - всё в одном посте
#     33327655_26089953 - отсебятина в Кирилловой полине + результаты выборов + хвост из комментов
#     и в последующих от первого созыва


Кандидаты 2 курс 2013-11-17
1
2
3
4
5
6
Кандидаты 3 курс 2013-11-17
1
2
3
4
5
6
7
8
Кандидаты 1 курс 2013-11-17
1
2
3
4
5
6
7
8
9
10
11
Кандидаты 4 курс 2013-11-17
1
2
3
4
5
6
7
8
9
10
Кандидаты 5 курс 2013-11-17
1
2
3
4
5
6
7
8
Кандидаты Магистратура 1 курс 2013-11-17
1
2
Кандидаты Магистратура 2 курс 2013-11-17
1
Кандидаты 1 курс 2012-11-04
1
2
3
4
5
6
Кандидаты 2 курс 2012-11-04
1
2
3
4
5
6
Кандидаты 3 курс 2012-11-04
1
2
3
4
5
Кандидаты 4 курс 2012-11-04
1
2
3
4
5
6
7
8
Кандидаты 5 курс 2012-11-04
1
2
Кандидаты, магистратура 2012-11-04
Кандидаты 5 курс 2012-03-14
1
2
3
Кандидаты 1 курс 2012-02-26
1
2
3
4
5
6
7
Кандидаты 4 курс 2012-02-25
1
2
3
4
5
Кандидаты 3 курс 2012-02-25
1
2
3
4
5
Кандидаты 2 курс 2012-02-25
1
2


{'https://vk.com/topic-33327655_29117899': {'title': 'Кандидаты 2 курс',
  'thread_message_count': 7,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17,
  'convocation_no': 3,
  'year_no': '2',
  'list_of_candidates': ['1 поток',
   '1. Дяченко Мария Игоревна, 204',
   '2. Остроухова Наталья Владимировна, 201',
   '3. Локтионов Фёдор Михайлович, 204',
   '2 поток',
   '1.Глунчадзе Ираклий Вахтангович, 207',
   '3 поток',
   '1. Ватутин Кирилл Александрович, 224',
   '1. Савченко Анастасия Игоревна, 224',
   'Ниже приведены рассказы кандидатов.'],
  'candidates': {'Дяченко Мария Игоревна': ['"Участвую в деятельности студсовета с осени 2012 года. Член групп по',
    'оповещению и образованию. Помогаю куратору 104 группы и всем',
    'первокурсникам, которые ко мне обращаются;) Вместе с Котляровым',
    'Никитой возила 103 группу на Посвят. Была ответственной за',
    'распределение повышенных стипендий весной 2013 у первого кур

In [369]:
# ручное удаление багов

# за 3 созыв
#     удалить коммент
del election_threads_dict['https://vk.com/topic-33327655_29117929']['candidates']["Знатный выбор у первого потока)"]

# за 2 созыв
#     неформатный м1 - https://vk.com/topic-33327655_27333041
election_threads_dict['https://vk.com/topic-33327655_27333041']['candidates'] = {
    election_threads_dict['https://vk.com/topic-33327655_27333041']['list_of_candidates'][1]: election_threads_dict['https://vk.com/topic-33327655_27333041']['list_of_candidates'][2:]
}
election_threads_dict['https://vk.com/topic-33327655_27333041']['list_of_candidates'] = election_threads_dict['https://vk.com/topic-33327655_27333041']['list_of_candidates'][:2]

# за 1 созыв
#     посты с итогами выборов
result_conv1_dict = {}
for link in election_threads_dict.keys():
    for name in election_threads_dict[link]['candidates'].keys():
        if 'Результаты выборов на ' in name:
            result_conv1_dict[name] = election_threads_dict[link]['candidates'][name]
#             del election_threads_dict[link]['candidates'][name]
            
election_threads_dict


{'https://vk.com/topic-33327655_29117899': {'title': 'Кандидаты 2 курс',
  'thread_message_count': 7,
  'link_date': datetime.date(2013, 11, 17),
  'link_year_no': 2013,
  'link_month_no': 11,
  'link_day_no': 17,
  'convocation_no': 3,
  'year_no': '2',
  'list_of_candidates': ['1 поток',
   '1. Дяченко Мария Игоревна, 204',
   '2. Остроухова Наталья Владимировна, 201',
   '3. Локтионов Фёдор Михайлович, 204',
   '2 поток',
   '1.Глунчадзе Ираклий Вахтангович, 207',
   '3 поток',
   '1. Ватутин Кирилл Александрович, 224',
   '1. Савченко Анастасия Игоревна, 224',
   'Ниже приведены рассказы кандидатов.'],
  'candidates': {'Дяченко Мария Игоревна': ['"Участвую в деятельности студсовета с осени 2012 года. Член групп по',
    'оповещению и образованию. Помогаю куратору 104 группы и всем',
    'первокурсникам, которые ко мне обращаются;) Вместе с Котляровым',
    'Никитой возила 103 группу на Посвят. Была ответственной за',
    'распределение повышенных стипендий весной 2013 у первого кур

In [54]:
# election_threads_dict['https://vk.com/topic-33327655_27333041']

In [ ]:
# ТЕПЕРЬ В ТАБЛИЦЫ

In [82]:
# чтобы выделить и удалить посты с результатами (в 1 созыве)

def isResult(name):
    if 'Результаты выборов на ' in name:
        return True
    else:
        return False

In [107]:
election_texts = {}

for link in election_threads_dict.keys():
    election_texts[link] = {
        'convocation_no': election_threads_dict[link]['convocation_no'],
        'year_no': election_threads_dict[link]['year_no'],
        'list_of_candidates': election_threads_dict[link]['list_of_candidates'],
        'candidates': election_threads_dict[link]['candidates']
    }
    
election_texts

{'https://vk.com/topic-33327655_29117899': {'convocation_no': 3,
  'year_no': '2',
  'list_of_candidates': ['1 поток',
   '1. Дяченко Мария Игоревна, 204',
   '2. Остроухова Наталья Владимировна, 201',
   '3. Локтионов Фёдор Михайлович, 204',
   '2 поток',
   '1.Глунчадзе Ираклий Вахтангович, 207',
   '3 поток',
   '1. Ватутин Кирилл Александрович, 224',
   '1. Савченко Анастасия Игоревна, 224',
   'Ниже приведены рассказы кандидатов.'],
  'candidates': {'Дяченко Мария Игоревна': ['"Участвую в деятельности студсовета с осени 2012 года. Член групп по',
    'оповещению и образованию. Помогаю куратору 104 группы и всем',
    'первокурсникам, которые ко мне обращаются;) Вместе с Котляровым',
    'Никитой возила 103 группу на Посвят. Была ответственной за',
    'распределение повышенных стипендий весной 2013 у первого курса и',
    'осенью 2013 у второго. Старалась вовремя доносить необходимую',
    'информацию до своего потока и курса.',
    'Хочу сделать еще больше, чтобы у людей не появл

In [278]:
# словарик для ручного исправления имен кандидатов

def fix_name(name):
    # incorrect: correct
    correct_names_dict = {
    #     из-за Е/Ё:
        'Кузнецова Алена': 'Кузнецова Алёна',
#         'Муравьева Анна Сергеевна': 'Муравьёва Анна Сергеевна',
    #     из-за опечаток:
        'Акушевич Андрей Дмитиевич': 'Акушевич Андрей Дмитриевич',
        'Ариас Альварес Владимир Владимирович': 'Ариас-Альварес Владимир Владимирович',
    #     из-за пропущенного отчества:
        'Сапунов Кирилл': 'Сапунов Кирилл Вячеславович',   
        'Матвеев Дмитрий': 'Матвеев Дмитрий Александрович',   
        'Власов Артем': 'Власов Артем Андреевич',   
        'Медведева Яна': 'Медведева Яна Григорьевна',   
        'Тишин Павел': 'Тишин Павел Владимирович',   
        'Сафина Диана': 'Сафина Диана Ураловна',   
        'Лапина София': 'Лапина Софья Михайловна',   
        'Чистяков Михаил': 'Чистяков Михаил Сергеевич',   
        'Денисов Илья': 'Денисов Илья Олегович',   
    #     из-за перепутанного отчества:
        'Котляров Никита Сергеевич': 'Котляров Никита Владимирович',   
    #     из-за перестановки слов:
        'Андрей Левин': 'Левин Андрей',   
    }
    if name in correct_names_dict.keys():
        return correct_names_dict[name]
    else:
        return name

fix_name('Андрей Левин')


'Левин Андрей'

In [284]:

announcement_names = {}
for link in election_texts.keys():
    for name in election_texts[link]['candidates'].keys():
        announcement_names[fix_name(clear_name(name))] = {
#             'link_year_no_x_name': str(announcement_links[link]['link_year_no']) + "_" + "_".join(fix_name(name).split()),
#             'link_year_no': announcement_links[link]['link_year_no'], 
#             'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
            'convocation_no': election_texts[link]['convocation_no'], 
            'year_no': election_texts[link]['year_no'], 
#             'stream_no': announcement_links[link]['nominated_info']['stream_no'], 
#             'academic_group_no': announcement_links[link]['nominated_info']['dict_nominations'][name]['academic_group_no'], 
            'program_txt': ' '.join(election_texts[link]['candidates'][name]),
            'link': link
        }

# announcement_names

announcement_names_df =  pd.DataFrame(announcement_names.items(), columns=['name', 'params'])
announcement_names_df_2 = pd.concat([announcement_names_df.drop(['params'], axis=1), announcement_names_df['params'].apply(pd.Series)], axis=1)
announcement_names_df_2 = announcement_names_df_2.sort_values(['convocation_no', 'year_no'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)

# announcement_names_df_2['name'] = announcement_names_df_2['name'].apply(lambda x: clear_name(x))
announcement_names_df_3 = announcement_names_df_2.drop(announcement_names_df_2[announcement_names_df_2['name'].apply(lambda x: isResult(x))].index, inplace=False).reset_index().drop(['index'], axis=1)
# announcement_names_df_3 = announcement_names_df_2

announcement_names_df_3[-60:]



,name,convocation_no,year_no,program_txt,link
12,Артамонова Мария Владимировна,1,4,"""Меня зовут Маша, я учусь на 4ом курсе на кафе...",https://vk.com/topic-33327655_26089874
13,Удимов Даниил Алексеевич,1,4,"""О себе и своих планах: Я староста 409 группы....",https://vk.com/topic-33327655_26089874
14,Чернодед Антон Игоревич,1,4,"""Давно хотел заняться общественной деятельност...",https://vk.com/topic-33327655_26089874
15,Штейников Юрий Николаевич,1,5,"""Ну и пару слов о себе и для чего мне это. Я у...",https://vk.com/topic-33327655_26191463
16,Савушкин Никита Максимович,1,5,"""Предсдатель старого ""Студенческого Совета мех...",https://vk.com/topic-33327655_26191463
17,Миценко Вадим Валериевич,1,5,"""Состою в ""старом студсовете"" в качестве замес...",https://vk.com/topic-33327655_26191463
18,Дяченко Мария Игоревна,2,1,"Привет! Меня зовут Дяченко Мария, я хотела бы ...",https://vk.com/topic-33327655_27333029
19,Остроухова Наталья Владимировна,2,1,Мне всегда была интересна общественная работа ...,https://vk.com/topic-33327655_27333029
20,Ватутин Кирилл Александрович,2,1,Могу подавать идеи чтоб делать жизнь в мгу инт...,https://vk.com/topic-33327655_27333029
21,Габидов Александр Сергеевич,2,1,"Я хотел бы вступить в Студенческий Совет, чтоб...",https://vk.com/topic-33327655_27333029


In [ ]:
# РАСПАРСИТЬ СПИСКИ КАНДИДАТОВ ИЗ list_of_candidates


In [285]:
nomination_dict = {}

for link in election_texts.keys():
    nomination_dict[link] = {
        'convocation_no': election_texts[link]['convocation_no'],
        'year_no': election_texts[link]['year_no'],
        'list_of_candidates': election_texts[link]['list_of_candidates']
    }
    
nomination_dict

{'https://vk.com/topic-33327655_29117899': {'convocation_no': 3,
  'year_no': '2',
  'list_of_candidates': ['1 поток',
   '1. Дяченко Мария Игоревна, 204',
   '2. Остроухова Наталья Владимировна, 201',
   '3. Локтионов Фёдор Михайлович, 204',
   '2 поток',
   '1.Глунчадзе Ираклий Вахтангович, 207',
   '3 поток',
   '1. Ватутин Кирилл Александрович, 224',
   '1. Савченко Анастасия Игоревна, 224',
   'Ниже приведены рассказы кандидатов.']},
 'https://vk.com/topic-33327655_29117864': {'convocation_no': 3,
  'year_no': '3',
  'list_of_candidates': ['1 поток',
   '1. Андреев Максим Александрович, 303',
   '2. Матвеев Дмитрий Александрович, 302',
   '2 поток',
   '1. Бунеев Артём Валерьевич, 310',
   '2. Борисов Василий Александрович, 310',
   '3. Хамматова Гульназ Марселевна, 312',
   '3 поток',
   '1. Быстрицкая Василина Васильевна, 321',
   '2. Медведева Яна Григорьевна, 321',
   '4 поток',
   '1. Хасанов Рафаэль Ильгизович, 331',
   '2. Заболотский Леонид Дмитриевич, 331',
   'Ниже приве

In [ ]:
# если нашел слово поток, то приписывай ему все строки до следующего слова поток (подобное было в соседнем ноуте)

# если есть слово поток, то берем оттуда число и тянем это число во все следующие строки, пока снова не найдется слово поток

In [286]:
list_of_candidates = ['1 поток',
   '1. Бурнус Артём Владимирович, 104',
   '2 поток',
   '1. Суркова Анастасия Юрьевна, 109',
   '2. Тюрина Татьяна Игоревна, 109',
   '3. Корно Дмитрий Сергеевич, 111',
   '4. Изимова Анна Мирбулатовна, 111',
   '5. Чистяков Михаил Сергеевич, 111',
   '6. Денисов Илья Олегович, 107',
   '3 поток',
   '1. Ариас Альварес Владимир Владимирович, 123',
   '2. Дорман Елена Вячеславовна, 126',
   '3. Григорьев Иван Сергеевич, 125',
   'Ниже приведены рассказы кандидатов.']

# НЕ ДЛЯ ВСЕХ ТИПОВ ПОДХОДИТ
def nomination_dict_by_list_conv3(list_of_candidates):
    stream_no = 0
    nom_dict = {}
    for line in list_of_candidates:
        if 'поток' in line:
            stream_no = re.sub('\D', '', line)
        elif 'Ниже приведены рассказы кандидатов' not in line:
            nom_dict[fix_name(clear_name(line))] = {
                'stream_no': stream_no,
                'academic_group_no': re.sub('\D', '', line[1:])
            }
    return nom_dict

nomination_dict_by_list_conv3(list_of_candidates)
# nomination_dict_by_list_conv3(nomination_dict['https://vk.com/topic-33327655_29117899']['list_of_candidates'])

# list_of_candidates = ['1 поток:',
#    '302 Осин Руслан Владимирович',
#    '303 Костицын Александр Алексеевич',
#    '306 Матвеев Алексей Викторович',
#    '2 поток:',
#    '307 Асташов Дмитрий Сергеевич',
#    '311 Антонов Сергей Валентинович',
#    '3 поток:',
#    '321 Смелов Владимир Павлович',
#    '321 Оджаев Рахмет Курбандурдыевич',
#    'Эконом-поток:',
#    '331 Лавров Василий Алексеевич']

# НЕ ДЛЯ ВСЕХ ТИПОВ ПОДХОДИТ
def nomination_dict_by_list_conv12(list_of_candidates):
    stream_no = 0
    nom_dict = {}
    for line in list_of_candidates:
        if 'поток' in line:
            stream_no = re.sub('\D', '', line)
            if 'эконом' in str.lower(line):
                stream_no = 4
        elif 'кандидатов' not in line and 'заявк' not in str.lower(line) and 'магистратур' not in str.lower(line):
            nom_dict[fix_name(clear_name(line))] = {
                'stream_no': stream_no,
                'academic_group_no': re.sub('\D', '', line)
            }
    return nom_dict

# nomination_dict_by_list_conv12(list_of_candidates)
# nomination_dict_by_list_conv12(nomination_dict['https://vk.com/topic-33327655_29117899']['list_of_candidates'])


def year_stream_no_by_year_stream(year,stream):
    if year in '123456' and stream in '12345':
        year_stream_no = year+'.'+stream
    else:
        year_stream_no = year
    return year_stream_no

# nomination_dict_by_list_conv3(list_of_candidates)


In [287]:
candidates_dict_123 = {}
for link in nomination_dict.keys():
    if nomination_dict[link]['convocation_no'] in [1,2]:
        nom_dict = nomination_dict_by_list_conv12(nomination_dict[link]['list_of_candidates'])
    elif nomination_dict[link]['convocation_no'] == 3:
        nom_dict = nomination_dict_by_list_conv3(nomination_dict[link]['list_of_candidates'])
    full_dict = {}
    for name in nom_dict.keys():
        full_dict[str(nomination_dict[link]['convocation_no']) + '_' + ''.join(name)] = {
            'name': name,
            'convocation_no': nomination_dict[link]['convocation_no'],
            'year_no': nomination_dict[link]['year_no'],
            'stream_no': nom_dict[name]['stream_no'],
            'academic_group_no': nom_dict[name]['academic_group_no'],
            'year_stream_no': year_stream_no_by_year_stream(nomination_dict[link]['year_no'],str(nom_dict[name]['stream_no'])),
            'link': link
        }
    candidates_dict_123 = {**candidates_dict_123, **full_dict}

candidates_dict_123


{'3_Дяченко Мария Игоревна': {'name': 'Дяченко Мария Игоревна',
  'convocation_no': 3,
  'year_no': '2',
  'stream_no': '1',
  'academic_group_no': '204',
  'year_stream_no': '2.1',
  'link': 'https://vk.com/topic-33327655_29117899'},
 '3_Остроухова Наталья Владимировна': {'name': 'Остроухова Наталья Владимировна',
  'convocation_no': 3,
  'year_no': '2',
  'stream_no': '1',
  'academic_group_no': '201',
  'year_stream_no': '2.1',
  'link': 'https://vk.com/topic-33327655_29117899'},
 '3_Локтионов Фёдор Михайлович': {'name': 'Локтионов Фёдор Михайлович',
  'convocation_no': 3,
  'year_no': '2',
  'stream_no': '1',
  'academic_group_no': '204',
  'year_stream_no': '2.1',
  'link': 'https://vk.com/topic-33327655_29117899'},
 '3_Глунчадзе Ираклий Вахтангович': {'name': 'Глунчадзе Ираклий Вахтангович',
  'convocation_no': 3,
  'year_no': '2',
  'stream_no': '2',
  'academic_group_no': '207',
  'year_stream_no': '2.2',
  'link': 'https://vk.com/topic-33327655_29117899'},
 '3_Ватутин Кирилл А

In [288]:
def name_df_by_name_dict(test_dict):
    df =  pd.DataFrame(test_dict.items(), columns=['conv_x_name', 'params'])
    df_2 = pd.concat([df.drop(['params'], axis=1), df['params'].apply(pd.Series)], axis=1)
    df_2 = df_2.sort_values(['convocation_no', 'year_no', 'stream_no', 'academic_group_no'],
                  ascending = [True, True, True, True]).reset_index().drop(['index'], axis=1)
#     df_2['name'] = df_2['name'].apply(lambda x: clear_name(x))
    return df_2

announcement_names_conv1_3 = name_df_by_name_dict(candidates_dict_123)
announcement_names_conv1_3[-60:]


,conv_x_name,name,convocation_no,year_no,stream_no,academic_group_no,year_stream_no,link
71,2_Тропин Александр Михайлович,Тропин Александр Михайлович,2,4,2,407,4.2,https://vk.com/topic-33327655_27333192
72,2_Коновалов Антон Александрович,Коновалов Антон Александрович,2,4,2,408,4.2,https://vk.com/topic-33327655_27333192
73,2_Сергеева Элина Александровна,Сергеева Элина Александровна,2,4,2,411,4.2,https://vk.com/topic-33327655_27333192
74,2_Ласкин Алексей Александрович,Ласкин Алексей Александрович,2,4,3,422,4.3,https://vk.com/topic-33327655_27333192
75,2_Тен Анна Бенхиевна,Тен Анна Бенхиевна,2,4,3,422,4.3,https://vk.com/topic-33327655_27333192
76,2_Акушевич Андрей Дмитриевич,Акушевич Андрей Дмитриевич,2,4,3,423,4.3,https://vk.com/topic-33327655_27333192
77,2_Дьяков Павел Александрович,Дьяков Павел Александрович,2,4,3,425,4.3,https://vk.com/topic-33327655_27333192
78,2_Сапунов Кирилл Вячеславович,Сапунов Кирилл Вячеславович,2,4,3,425,4.3,https://vk.com/topic-33327655_27333192
79,2_Федорова Елена Игоревна,Федорова Елена Игоревна,2,5,4,532,5.4,https://vk.com/topic-33327655_27333038
80,2_Абдуллаева Эмилия Табриз кызы,Абдуллаева Эмилия Табриз кызы,2,5,1,505,5.1,https://vk.com/topic-33327655_27333038


In [289]:
def fix_magister_year(year_no):
    if year_no in ['m1', 'm2']:
        return int(year_no[-1])
    else:
        return int(year_no)

announcement_names_conv_1_3_full = announcement_names_conv1_3[['name','convocation_no','year_stream_no','year_no','stream_no','academic_group_no','link']].merge(announcement_names_df_3[['name','convocation_no','program_txt']], left_on=['name','convocation_no'], right_on=['name','convocation_no'],how='outer')
announcement_names_conv_1_3_full['year_no'] = announcement_names_conv_1_3_full['year_no'].apply(lambda x: fix_magister_year(x))
# announcement_names_conv_1_3_full[0:60]
# announcement_names_conv_1_3_full[60:120]
announcement_names_conv_1_3_full[-60:]


# + ФЕДОРОВА в 1 созыве!!! видно по результатам


,name,convocation_no,year_stream_no,year_no,stream_no,academic_group_no,link,program_txt
71,Тропин Александр Михайлович,2,4.2,4,2,407,https://vk.com/topic-33327655_27333192,NaN
72,Коновалов Антон Александрович,2,4.2,4,2,408,https://vk.com/topic-33327655_27333192,NaN
73,Сергеева Элина Александровна,2,4.2,4,2,411,https://vk.com/topic-33327655_27333192,"""Я учусь на 4ом курсе на кафедре МаТИС. Я бы х..."
74,Ласкин Алексей Александрович,2,4.3,4,3,422,https://vk.com/topic-33327655_27333192,NaN
75,Тен Анна Бенхиевна,2,4.3,4,3,422,https://vk.com/topic-33327655_27333192,"""Помощь в подготовке и проведении дня мехмата."""
76,Акушевич Андрей Дмитриевич,2,4.3,4,3,423,https://vk.com/topic-33327655_27333192,"""Являюсь активном членом студенческого отряда ..."
77,Дьяков Павел Александрович,2,4.3,4,3,425,https://vk.com/topic-33327655_27333192,"""Являюсь активном членом студенческого отряда ..."
78,Сапунов Кирилл Вячеславович,2,4.3,4,3,425,https://vk.com/topic-33327655_27333192,NaN
79,Федорова Елена Игоревна,2,5.4,5,4,532,https://vk.com/topic-33327655_27333038,NaN
80,Абдуллаева Эмилия Табриз кызы,2,5.1,5,1,505,https://vk.com/topic-33327655_27333038,рассказ о себе и своих планах работы в СтудСов...


In [290]:
announcement_names_conv_1_3_full.to_csv('ssmm_candidates_info_conv_1_3.csv', sep='\t', encoding='utf-8', index=False)


## про результаты

In [ ]:
# ОСТАЛОСЬ РАСПАРСИТЬ РЕЗУЛЬТАТЫ

In [291]:
# 1 созыв
result_conv1_dict

{'Результаты выборов на первом курсе:': ['Первый поток:',
  'Матвеев Д. - 49 (голосов)',
  'Левин А. - 48',
  'против всех - 6',
  'количество проголосовавших - 68 (напоминаю, что студенты имели право проголосовать за 0, 1, 2 или 3 кандидатов).',
  'численность потока - 139.',
  'достигнутый кворум - 49%',
  'итог: представителями 1 потока 1 курса в Студсовете мехмата выбраны оба студента.',
  'Второй поток:',
  'Кириллова П. - 40',
  'Власов А. - 36',
  'Кузнецова А. - 28',
  'Семенов П. - 20',
  'против всех -2',
  'количество проголосовавших - 60',
  'численность потока - 138',
  'достигнутый кворум - 43%',
  'итог: представителями 2 потока 1 курса в Студсовете мехмата выбраны 3 студента с наибольшим числом голосов.',
  'Третий поток:',
  'Тишин П. - 51',
  'Сафина Д. - 44',
  'Медведева Я. - 23',
  'Кунденко В. - 21',
  'Скобенникова Ю. - 17',
  'против всех - 5',
  'количество проголосовавших - 73',
  'численность потока - 115',
  'достигнутый кворум - 63%',
  'итог: представителя

In [336]:
def no_by_text(line):
    line1 = str.lower(line)
    if 'перв' in line1:
        year_no = 1
    elif 'втор' in line1:
        year_no = 2
    elif 'трет' in line1:
        year_no = 3
    elif 'четв' in line1 or 'эконом' in line1:
        year_no = 4
    else:
        year_no = -1
    return year_no



# ОТДЕЛЬНО ПРО КАНДИДАТОВ

conv1_pers_result_dict = {}
for course in result_conv1_dict.keys():
    year_no = no_by_text(course)
    for line in result_conv1_dict[course]:
#         while not ('курса выбраны' in line):
        if no_by_text(line) > 0:
            stream_no = no_by_text(line)
    #            print(no_by_text(line))
        elif ('курса выбраны' in line):
            break
        elif not (any(word in line for word in ['против всех', 'количество', 'численность', 'итог', 'кворум', 'результат', 'выбор', 'голосован'])):
    #     elif not ('против всех' in line or 'количество' in line or 'численность' in line or 'итог' in line or 'кворум' in line):
            conv1_pers_result_dict[line] = {
    #                 'full_line': line,
                'name': fix_name(clear_name(line)),
                'convocation_no': 1,
                'year_no': year_no,
                'stream_no': stream_no,
                'votes_cnt': re.sub('\D', '', line)
            }
#                 conv1_result_dict = {**conv1_result_dict, **conv1_result_dict}
conv1_pers_result_dict



{'Матвеев Д. - 49 (голосов)': {'name': 'Матвеев Д',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 1,
  'votes_cnt': '49'},
 'Левин А. - 48': {'name': 'Левин А',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 1,
  'votes_cnt': '48'},
 'Кириллова П. - 40': {'name': 'Кириллова П',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '40'},
 'Власов А. - 36': {'name': 'Власов А',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '36'},
 'Кузнецова А. - 28': {'name': 'Кузнецова А',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '28'},
 'Семенов П. - 20': {'name': 'Семенов П',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '20'},
 'Тишин П. - 51': {'name': 'Тишин П',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 3,
  'votes_cnt': '51'},
 'Сафина Д. - 44': {'name': 'Сафина Д',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 3,
  'votes_cnt': '44'},
 'Медведева Я. - 23': {'na

In [337]:
import copy

elected_names = {}
votes_names = copy.deepcopy(conv1_pers_result_dict)

for name in conv1_pers_result_dict.keys():
    if re.findall(r'[\d]{3}\s\-\s[\w]+', name):
        elected_names[name] = {
            'name': fix_name(clear_name(name)),
            'academic_group_no': conv1_pers_result_dict[name]['votes_cnt']
        } 
        del votes_names[name]
        
elected_names

{'101 - Левин Андрей;': {'name': 'Левин Андрей', 'academic_group_no': '101'},
 '101 - Матвеев Дмитрий;': {'name': 'Матвеев Дмитрий Александрович',
  'academic_group_no': '101'},
 '107 - Власов Артем;': {'name': 'Власов Артем Андреевич',
  'academic_group_no': '107'},
 '112 - Кириллова Полина;': {'name': 'Кириллова Полина',
  'academic_group_no': '112'},
 '112 - Кузнецова Алёна;': {'name': 'Кузнецова Алёна',
  'academic_group_no': '112'},
 '121 - Медведева Яна;': {'name': 'Медведева Яна Григорьевна',
  'academic_group_no': '121'},
 '122 - Тишин Павел;': {'name': 'Тишин Павел Владимирович',
  'academic_group_no': '122'},
 '125 - Сафина Диана.': {'name': 'Сафина Диана Ураловна',
  'academic_group_no': '125'},
 '402 - Артамонова Мария Владимировна': {'name': 'Артамонова Мария Владимировна',
  'academic_group_no': '402'},
 '402 - Цылин Иван Вячеславович': {'name': 'Цылин Иван Вячеславович',
  'academic_group_no': '402'},
 '405 - Абдуллаева Эмилия Табриз кызы': {'name': 'Абдуллаева Эмилия Та

In [339]:
elected_names_df =  pd.DataFrame(elected_names.items(), columns=['group_x_name', 'params'])
elected_names_df_2 = pd.concat([elected_names_df.drop(['params'], axis=1), elected_names_df['params'].apply(pd.Series)], axis=1)
elected_names_df_2 = elected_names_df_2.sort_values(['academic_group_no'],
                  ascending = [True]).reset_index().drop(['index'], axis=1)
elected_names_df_2


,group_x_name,name,academic_group_no
0,101 - Левин Андрей;,Левин Андрей,101
1,101 - Матвеев Дмитрий;,Матвеев Дмитрий Александрович,101
2,107 - Власов Артем;,Власов Артем Андреевич,107
3,112 - Кириллова Полина;,Кириллова Полина,112
4,112 - Кузнецова Алёна;,Кузнецова Алёна,112
5,121 - Медведева Яна;,Медведева Яна Григорьевна,121
6,122 - Тишин Павел;,Тишин Павел Владимирович,122
7,125 - Сафина Диана.,Сафина Диана Ураловна,125
8,202 - Гильманов Роман Павлович;,Гильманов Роман Павлович,202
9,205 - Шариков Алексей Анатольевич,Шариков Алексей Анатольевич,205


In [340]:


votes_names['Федорову Елену Игоревну'] = {
    'name': 'Федорова Елена Игоревна',
    'convocation_no': 1,
    'year_no': 4,
    'stream_no': 4,
    'votes_cnt': '20'
}
# 'За кандидата Федорову Елену Игоревну проголосовали 20 студентов из 32, кворум - 63%': {'name': 'За кандидата Федорову Елену Игоревну проголосовали'
# ДОБАВИТЬ РУКАМИ

votes_names

{'Матвеев Д. - 49 (голосов)': {'name': 'Матвеев Д',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 1,
  'votes_cnt': '49'},
 'Левин А. - 48': {'name': 'Левин А',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 1,
  'votes_cnt': '48'},
 'Кириллова П. - 40': {'name': 'Кириллова П',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '40'},
 'Власов А. - 36': {'name': 'Власов А',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '36'},
 'Кузнецова А. - 28': {'name': 'Кузнецова А',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '28'},
 'Семенов П. - 20': {'name': 'Семенов П',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 2,
  'votes_cnt': '20'},
 'Тишин П. - 51': {'name': 'Тишин П',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 3,
  'votes_cnt': '51'},
 'Сафина Д. - 44': {'name': 'Сафина Д',
  'convocation_no': 1,
  'year_no': 1,
  'stream_no': 3,
  'votes_cnt': '44'},
 'Медведева Я. - 23': {'na

In [341]:
votes_names_df =  pd.DataFrame(votes_names.items(), columns=['group_x_name', 'params'])
votes_names_df_2 = pd.concat([votes_names_df.drop(['params'], axis=1), votes_names_df['params'].apply(pd.Series)], axis=1)
# votes_names_df_2['elected_flg'] = 0
votes_names_df_2 = votes_names_df_2.sort_values(['year_no','stream_no'],
                  ascending = [True,True]).reset_index().drop(['index'], axis=1)
votes_names_df_2


,group_x_name,name,convocation_no,year_no,stream_no,votes_cnt
0,Матвеев Д. - 49 (голосов),Матвеев Д,1,1,1,49
1,Левин А. - 48,Левин А,1,1,1,48
2,Кириллова П. - 40,Кириллова П,1,1,2,40
3,Власов А. - 36,Власов А,1,1,2,36
4,Кузнецова А. - 28,Кузнецова А,1,1,2,28
5,Семенов П. - 20,Семенов П,1,1,2,20
6,Тишин П. - 51,Тишин П,1,1,3,51
7,Сафина Д. - 44,Сафина Д,1,1,3,44
8,Медведева Я. - 23,Медведева Я,1,1,3,23
9,Кунденко В. - 21,Кунденко В,1,1,3,21


In [349]:
def short_name_by_full_name(full_name):
    names = list(votes_names_df_2['name'])
    for short_name in names:
        if short_name in full_name:
            return short_name

# short_name_by_full_name('Котляров Никита Владимирович')

elected_names_df_3 = elected_names_df_2
elected_names_df_3['short_name'] = elected_names_df_3['name'].apply(lambda x: short_name_by_full_name(x))
elected_names_df_3['elected_flg'] = 1
elected_names_df_3


,group_x_name,name,academic_group_no,short_name,elected_flg
0,101 - Левин Андрей;,Левин Андрей,101,Левин А,1
1,101 - Матвеев Дмитрий;,Матвеев Дмитрий Александрович,101,Матвеев Д,1
2,107 - Власов Артем;,Власов Артем Андреевич,107,Власов А,1
3,112 - Кириллова Полина;,Кириллова Полина,112,Кириллова П,1
4,112 - Кузнецова Алёна;,Кузнецова Алёна,112,Кузнецова А,1
5,121 - Медведева Яна;,Медведева Яна Григорьевна,121,Медведева Я,1
6,122 - Тишин Павел;,Тишин Павел Владимирович,122,Тишин П,1
7,125 - Сафина Диана.,Сафина Диана Ураловна,125,Сафина Д,1
8,202 - Гильманов Роман Павлович;,Гильманов Роман Павлович,202,Гильманов Р,1
9,205 - Шариков Алексей Анатольевич,Шариков Алексей Анатольевич,205,Шариков А,1


In [357]:
# votes_names_df_2
# name	convocation_no	year_no	stream_no	votes_cnt
# elected_names_df_2
# name	academic_group_no

results_candidates_conv1 = votes_names_df_2[['name','convocation_no','year_no','stream_no','votes_cnt']].merge(elected_names_df_3[['short_name','name','academic_group_no','elected_flg']], left_on = 'name', right_on = 'short_name', how = 'outer')
results_candidates_conv1['max_name'] = results_candidates_conv1['name_y'].combine_first(results_candidates_conv1['name_x'])
results_candidates_conv1 = results_candidates_conv1.fillna(value={'elected_flg': 0})
results_candidates_conv1 = results_candidates_conv1[['max_name','convocation_no','year_no','stream_no','academic_group_no','votes_cnt','elected_flg']].rename(columns={'max_name':'name'})
results_candidates_conv1



,name,convocation_no,year_no,stream_no,academic_group_no,votes_cnt,elected_flg
0,Матвеев Дмитрий Александрович,1,1,1,101,49,1.0
1,Левин Андрей,1,1,1,101,48,1.0
2,Кириллова Полина,1,1,2,112,40,1.0
3,Власов Артем Андреевич,1,1,2,107,36,1.0
4,Кузнецова Алёна,1,1,2,112,28,1.0
5,Семенов П,1,1,2,NaN,20,0.0
6,Тишин Павел Владимирович,1,1,3,122,51,1.0
7,Сафина Диана Ураловна,1,1,3,125,44,1.0
8,Медведева Яна Григорьевна,1,1,3,121,23,1.0
9,Кунденко В,1,1,3,NaN,21,0.0


In [358]:
results_candidates_conv1.to_csv('ssmm_votes_info_conv_1.csv', sep='\t', encoding='utf-8', index=False)


In [374]:

# ОТДЕЛЬНО ПРО СТАТУ ПОТОКА (против всех итд)

conv1_stream_result_dict = {}
for course in result_conv1_dict.keys():
    year_no = no_by_text(course)
    for line in result_conv1_dict[course]:
        if no_by_text(line) > 0 and not('Представителями ' in line):
            stream_no = no_by_text(line)
            conv1_stream_result_dict[str(year_no)+'.'+str(stream_no)] = {}
        elif 'против всех' in line:
            conv1_stream_result_dict[str(year_no)+'.'+str(stream_no)]['NOTA_cnt'] = int(re.sub('\D', '', line))
        elif 'численность' in line and not ('общая' in line):
            conv1_stream_result_dict[str(year_no)+'.'+str(stream_no)]['stream_size'] = int(re.sub('\D', '', line))
        elif 'количество проголосовавших' in line and not ('на выборах' in line):
            if '0123' in re.sub('\D', '', line):
                conv1_stream_result_dict[str(year_no)+'.'+str(stream_no)]['voted_cnt'] = int(re.sub('\D', '', line)[:-4])
            else:
                conv1_stream_result_dict[str(year_no)+'.'+str(stream_no)]['voted_cnt'] = int(re.sub('\D', '', line))
            
conv1_stream_result_dict


# РУКАМИ то, что не автоматизируется:


conv1_stream_result_dict['2.3']['NOTA_cnt'] = 0

#   'За предоставленное время не было подано ни одной заявки. На данный момент собрана 21 подпись из общей численности 29 под решением оставить экономпоток без представителя в Студсовете.',
conv1_stream_result_dict['3.4']['NOTA_cnt'] = 21
conv1_stream_result_dict['3.4']['voted_cnt'] = 21
conv1_stream_result_dict['3.4']['stream_size'] = 29

# 'За кандидата Федорову Елену Игоревну проголосовали 20 студентов из 32, кворум - 63%': {'name': 'За кандидата Федорову Елену Игоревну проголосовали'
conv1_stream_result_dict['4.4']['NOTA_cnt'] = 0
conv1_stream_result_dict['4.4']['voted_cnt'] = 20
conv1_stream_result_dict['4.4']['stream_size'] = 32

conv1_stream_result_dict


{'1.1': {'NOTA_cnt': 6, 'voted_cnt': 68, 'stream_size': 139},
 '1.2': {'NOTA_cnt': 2, 'voted_cnt': 60, 'stream_size': 138},
 '1.3': {'NOTA_cnt': 5, 'voted_cnt': 73, 'stream_size': 115},
 '4.1': {'NOTA_cnt': 1, 'voted_cnt': 69, 'stream_size': 97},
 '4.2': {'NOTA_cnt': 2, 'voted_cnt': 49, 'stream_size': 94},
 '4.3': {'NOTA_cnt': 1, 'voted_cnt': 48, 'stream_size': 93},
 '4.4': {'NOTA_cnt': 0, 'voted_cnt': 20, 'stream_size': 32},
 '3.1': {'NOTA_cnt': 2, 'voted_cnt': 64, 'stream_size': 119},
 '3.2': {'NOTA_cnt': 0, 'voted_cnt': 62, 'stream_size': 113},
 '3.3': {'NOTA_cnt': 0, 'voted_cnt': 46, 'stream_size': 81},
 '3.4': {'NOTA_cnt': 21, 'voted_cnt': 21, 'stream_size': 29},
 '2.1': {'NOTA_cnt': 5, 'voted_cnt': 91, 'stream_size': 133},
 '2.2': {'NOTA_cnt': 3, 'voted_cnt': 60, 'stream_size': 111},
 '2.3': {'voted_cnt': 71, 'stream_size': 113, 'NOTA_cnt': 0}}

In [375]:
stream_results_df =  pd.DataFrame(conv1_stream_result_dict.items(), columns=['year_stream_no', 'params'])
stream_results_df_2 = pd.concat([stream_results_df.drop(['params'], axis=1), stream_results_df['params'].apply(pd.Series)], axis=1)
# votes_names_df_2['elected_flg'] = 0
stream_results_df_2 = stream_results_df_2.sort_values(['year_stream_no'],
                  ascending = [True]).reset_index().drop(['index'], axis=1)
stream_results_df_2


,year_stream_no,NOTA_cnt,voted_cnt,stream_size
0,1.1,6,68,139
1,1.2,2,60,138
2,1.3,5,73,115
3,2.1,5,91,133
4,2.2,3,60,111
5,2.3,0,71,113
6,3.1,2,64,119
7,3.2,0,62,113
8,3.3,0,46,81
9,3.4,21,21,29


In [376]:
stream_results_df_2.to_csv('ssmm_results_info_conv_1.csv', sep='\t', encoding='utf-8', index=False)


## ТЕСТИРОВАНИЕ

In [ ]:
'''
ССЫЛКИ НА ТРЕДЫ С КАНДИДАТАМИ:
-
'''

announcements_links_threads = {
    'conv1': {
        '1': 'https://vk.com/topic-33327655_26089953',
        '2': 'https://vk.com/topic-33327655_26089854',
        '3': 'https://vk.com/topic-33327655_26089737',
        '4': 'https://vk.com/topic-33327655_26089874',
        '5': 'https://vk.com/topic-33327655_26191463'  # нет итогов
    },
    'conv2': {  # нет итогов
        '1': 'https://vk.com/topic-33327655_27333029',
        '2': 'https://vk.com/topic-33327655_27333032',
        '3': 'https://vk.com/topic-33327655_27333034',
        '4': 'https://vk.com/topic-33327655_27333192',
        '5': 'https://vk.com/topic-33327655_27333038',
        'm1': 'https://vk.com/topic-33327655_27333041'
    },
    'conv3': { # итоги в посте: https://vk.com/wall-33327655_1600
        '1': 'https://vk.com/topic-33327655_29117929',
        '2': 'https://vk.com/topic-33327655_29117899',
        '3': 'https://vk.com/topic-33327655_29117864',
        '4': 'https://vk.com/topic-33327655_29117826',
        '5': 'https://vk.com/topic-33327655_29117817',
        'm1': 'https://vk.com/topic-33327655_29117788',
        'm2': 'https://vk.com/topic-33327655_29117770'
    }
}
announcements_links_threads


In [67]:
req_announcement = requests.get('https://vk.com/topic-33327655_29117899')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ka" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="1 поток 1. Дяченко Мария Игоревна, 204 2. Остроухова Наталья Владимировна, 201 3. Локтионов Фёдор Михайлович, 204" />
        <meta property="og:description" content="1 поток 1. Дяченко Мария Игоревна, 204 2. Остроухова Наталья Владимировна, 201 3. Локтионов Фёдор Михайлович, 204" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav

# СТАРОЕ

In [ ]:
# for link in thread_dict:
#     print(thread_dict[link]['thread_title'])
#     for sheet in range(thread_dict[link]['thread_message_count']//20 + 1):
#         print(thread_dict[link]['thread_link'] + '?offset=' + str(sheet*20))
#         print(vk_thread_info_by_link(thread_dict[link]['thread_link'] + '?offset=' + str(sheet*20)))
        
# ОШИБКА: В ЭТОМ ТРЕДЕ НЕТ БЛОКОВ С ТЕКСТОМ

In [83]:
# https://vk.com/board33327655

# req_announcement = requests.get('https://vk.com/board33327655')
# html_doc = req_announcement.text
# print(html_doc)

# только верхние 20 тем


In [82]:
# req_announcement = requests.get('https://vk.com/board33327655?offset=20') # так можно листать по 20 тем, норм
# html_doc = req_announcement.text
# print(html_doc)

In [93]:
# В ЭТОМ ТРЕДЕ НЕТ БЛОКОВ С ТЕКСТОМ
req_announcement = requests.get('https://vk.com/topic-33327655_47378210')
html_doc = req_announcement.text
print(html_doc)

# МОЖНО ВЫТАЩИТЬ АДРЕС ССЫЛКИ НА ФАЙЛ, ОН РАБОТАЕТ
# /doc121940016_590903120?hash=44d29fbd96c2702270&dl=b1be48be63b6777292

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="Здесь находятся протоколы собраний Студенческого совета мехмата, начиная с X созыва." />
        <meta property="og:description" content="Здесь находятся протоколы собраний Студенческого совета мехмата, начиная с X созыва." />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" 

In [85]:
test_link = 'https://vk.com/topic-33327655_26170925?offset=20'

test_dict = vk_thread_info_by_link(test_link)
print(test_dict)

{'https://vk.com/topic-33327655_26170925?offset=20': {'list_of_posts': {0: {'link_date': datetime.date(2012, 3, 21), 'link_year_no': 2012, 'link_month_no': 3, 'link_day_no': 21, 'link_date_time': '21 мар 2012 в 11:43', 'link_time': '11:43', 'author': 'Александр Тропин', 'list_of_lines': ['у встреч рабгрупп сейчас есть один общий недостаток - всем удобно собираться по вторникам в одно и то же время)', 'предлагаю составлять текст обращения к ректору по открытию перехода. я узнаю подробности, как действовала ИГ зимой 2010-го, когда открыли. то есть, если что, предупредить ректора, что будут сотни подписей мехматян, если одного письма от студсовета будет недостаточно.']}, 1: {'link_date': datetime.date(2012, 3, 30), 'link_year_no': 2012, 'link_month_no': 3, 'link_day_no': 30, 'link_date_time': '30 мар 2012 в 1:10', 'link_time': '1:10', 'author': 'Александр Тропин', 'list_of_lines': ['кстати, в продолжение темы.', 'в архивах форумлокала наткнулся сегодня на забавный пост: стенограмма встреч

In [14]:
link_vk = 'https://vk.com/topic-33327655_25816532'
req_link = requests.get(link_vk)
html_doc = req_link.text
soup = BeautifulSoup(html_doc, 'html.parser')

wi_body = soup.find_all('div', attrs={'class': 'pi_body'})
# print(wi_body)

for el in wi_body:
    text = el.find('div', attrs={'class': 'pi_text'})
#     print(text)
    text_lines = [line for line in text.stripped_strings]
    print(text_lines)
    
    date = 

['На собрании присутствовало 9 представителей студсовета. Модерировал собрание Никита Котляров, выбранный председателем на экстренном собрании в декабре. Собрание длилось около 3 часов, модерирование было затруднено в связи с большим желанием участников обсудить следующие пункты повестки:', '1) формат собрания.', 'Обсудили, как будут проходить наши собрания в будущем. Модератор объявляет повестку собрания, далее по пунктам участники высказываются в порядке очереди по поднятой руке. На собраниях могут присутствовать все желающие (не только студсоветовцы, но и остальные студенты), а преподаватели и представители администрации - либо по приглашению студсоветовцев, либо при отсутствии возражений у присутствующих на собрании студсоветовцев.', '2) Устав.', 'а) выборы.', 'Уже в феврале-марте нужно организовать перевыборы ССовета мехмата. Единогласно были приняты следующие пункты:', '- перевыборы проходят ежегодно, в феврале-марте (первокурсники успеют познакомиться, сдать первую сессию).', '-

In [16]:
# если ссылка некорректная, то происходит переадрессация на https://vk.com/board33327655

# req_announcement = requests.get('https://vk.com/topic-33327655_25816533')
# html_doc = req_announcement.text
# print(html_doc)